# TDP testing notebook

## Jupyter Question 1

In [24]:
from cdapython import Q, unique_terms, columns
import requests
import pprint

<font color = blue>Adding the direct query since some of the python results seem odd</font>

In [3]:
def runAPIQuery(querystring, limit):
    #Using a limit:
    if limit is not None:
        cdaURL = "https://cda.cda-dev.broadinstitute.org/api/v1/sql-query/v3?limit={}".format(str(limit))
    else:
        cdaURL = 'https://cda.cda-dev.broadinstitute.org/api/v1/sql-query/v3'
    headers = {'accept' : 'application/json', 'Content-Type' : 'text/plain'}

    request = requests.post(cdaURL, headers = headers, data = querystring)

    if request.status_code == 200:
        return request.json()
    else:
        raise Exception ("Query failed code {}. {}".format(request.status_code,query))

### TEST QUERY:  Find data from TCGA-BRCA with donors over 50 and stage IIIC cancer

<font color=blue> Is "stage IIIC cancer" tumor stage or tumor grade? </font>

In [4]:
unique_terms('ResearchSubject.Diagnosis.tumor_grade')

SELECT DISTINCT(_Diagnosis.tumor_grade) FROM `gdc-bq-sample.cda_mvp.v3`, UNNEST(ResearchSubject) AS _ResearchSubject,UNNEST(_ResearchSubject.Diagnosis) AS _Diagnosis ORDER BY _Diagnosis.tumor_grade


[None,
 'G1',
 'G2',
 'G3',
 'G4',
 'GB',
 'GX',
 'High Grade',
 'Low Grade',
 'Not Reported',
 'Unknown',
 'not reported']

In [5]:
unique_terms('ResearchSubject.Diagnosis.tumor_stage')

SELECT DISTINCT(_Diagnosis.tumor_stage) FROM `gdc-bq-sample.cda_mvp.v3`, UNNEST(ResearchSubject) AS _ResearchSubject,UNNEST(_ResearchSubject.Diagnosis) AS _Diagnosis ORDER BY _Diagnosis.tumor_stage


[None,
 '',
 '1B',
 '2A',
 '2B',
 '3',
 '4',
 'Adverse',
 'Favorable',
 'FavorableOrIntermediate',
 'I',
 'I b',
 'IA',
 'IB',
 'IC',
 'II',
 'II b',
 'IIA',
 'IIB',
 'III',
 'IIIA',
 'IIIB',
 'IIIC',
 'IPI:0',
 'IPI:1',
 'IPI:12',
 'IPI:13',
 'IPI:14',
 'IPI:15',
 'IPI:2',
 'IPI:23',
 'IPI:24',
 'IPI:25',
 'IPI:3',
 'IPI:34',
 'IPI:35',
 'IPI:4',
 'IPI:45',
 'IPI:5',
 'IV',
 'IVA',
 'IVB',
 'IVa',
 'IVb',
 'Intermediate',
 'IntermediateOrAdverse',
 'N/A',
 'Normal',
 'Not Performed',
 'Not Reported',
 'Not Reported/ Unknown',
 'Not Reported/Unknown',
 'PT4apN0',
 'Stage 1B',
 'Stage I',
 'Stage IA',
 'Stage IA3',
 'Stage IB',
 'Stage IC',
 'Stage II',
 'Stage IIA',
 'Stage IIB',
 'Stage III',
 'Stage IIIA',
 'Stage IIIB',
 'Stage IIIC',
 'Stage IV',
 'Stage IVA',
 'Stage IVB',
 'Stage1',
 'T1N0Mx',
 'T1aN0M0',
 'T2',
 'TxNxM1',
 'Unknown',
 'i',
 'i/ii nos',
 'ii',
 'ii/v',
 'iii',
 'iii/v',
 'iiib',
 'iiib/v',
 'is',
 'iv',
 'iv/v',
 'na',
 'no resection',
 'not reported',
 'pT1',
 '

<font color = blue> Clearly tumor_stage, however there are three options:  *IIIC*, *Stage IIIC* and *stage iiic*</font>

In [6]:
q1 = Q('ResearchSubject.associated_project = "TCGA-BRCA"')
q2 = Q('ResearchSubject.Diagnosis.age_at_diagnosis > 50*365')
g1 = Q('ResearchSubject.Diagnosis.tumor_stage = "IIIC"')
g2 = Q('ResearchSubject.Diagnosis.tumor_stage = "Stage IIIC"')
g3 = Q('ResearchSubject.Diagnosis.tumor_stage = "stage iiic"')

<font color = blue> So test the tumor stage differences one at a time </font>

In [7]:
q = q1.And(q2.And(g1))
r1 = q.run()
print(r1)


Query: SELECT * FROM gdc-bq-sample.cda_mvp.v3, UNNEST(ResearchSubject) AS _ResearchSubject, UNNEST(_ResearchSubject.Diagnosis) AS _Diagnosis WHERE ((_ResearchSubject.associated_project = 'TCGA-BRCA') AND ((_Diagnosis.age_at_diagnosis > 50*365) AND (_Diagnosis.tumor_stage = 'IIIC')))
Offset: 0
Limit: 1000
Count: 0
More pages: No



In [8]:
q = q1.And(q2.And(g2))
r2 = q.run()
print(r2)


Query: SELECT * FROM gdc-bq-sample.cda_mvp.v3, UNNEST(ResearchSubject) AS _ResearchSubject, UNNEST(_ResearchSubject.Diagnosis) AS _Diagnosis WHERE ((_ResearchSubject.associated_project = 'TCGA-BRCA') AND ((_Diagnosis.age_at_diagnosis > 50*365) AND (_Diagnosis.tumor_stage = 'Stage IIIC')))
Offset: 0
Limit: 1000
Count: 0
More pages: No



In [9]:
q = q1.And(q2.And(g3))
r3 = q.run()
print(r3)


Query: SELECT * FROM gdc-bq-sample.cda_mvp.v3, UNNEST(ResearchSubject) AS _ResearchSubject, UNNEST(_ResearchSubject.Diagnosis) AS _Diagnosis WHERE ((_ResearchSubject.associated_project = 'TCGA-BRCA') AND ((_Diagnosis.age_at_diagnosis > 50*365) AND (_Diagnosis.tumor_stage = 'stage iiic')))
Offset: 0
Limit: 1000
Count: 45
More pages: No



<font color = blue> From these results, clearly *stage iiic* is the terminology used </font>

In [10]:
for patient in r3:
    for researchsubject in patient['ResearchSubject']:
        for rs in researchsubject['Diagnosis']:
            print(("Stage: %s\tGrade: %s\tPrimary Diagnosis: %s\n") % (rs['tumor_stage'], rs['tumor_grade'], rs['primary_diagnosis']))

Stage: stage iiic	Grade: not reported	Primary Diagnosis: Pleomorphic carcinoma

Stage: stage iiic	Grade: not reported	Primary Diagnosis: Infiltrating duct carcinoma, NOS

Stage: stage iiic	Grade: not reported	Primary Diagnosis: Infiltrating duct carcinoma, NOS

Stage: stage iiic	Grade: not reported	Primary Diagnosis: Infiltrating duct carcinoma, NOS

Stage: stage iiic	Grade: not reported	Primary Diagnosis: Infiltrating duct carcinoma, NOS

Stage: stage iiic	Grade: not reported	Primary Diagnosis: Infiltrating duct carcinoma, NOS

Stage: stage iiic	Grade: not reported	Primary Diagnosis: Infiltrating duct carcinoma, NOS

Stage: stage iiic	Grade: not reported	Primary Diagnosis: Infiltrating duct carcinoma, NOS

Stage: stage iiic	Grade: not reported	Primary Diagnosis: Infiltrating duct carcinoma, NOS

Stage: stage iiic	Grade: not reported	Primary Diagnosis: Infiltrating duct carcinoma, NOS

Stage: stage iiic	Grade: not reported	Primary Diagnosis: Infiltrating duct carcinoma, NOS

Stage: sta

<font color=blue> So then the last step would be to see what data are associcated wtih these cases.  It looks like either File.data_type or File.data_category would do<font>

In [ ]:
for patient in r3:
    for subject in patient['ResearchSubject']:
        for specimen in subject['Specimen']:
            for file in specimen['File']:
                print(("Data Type: %s\t\tData Category: %s\n") % (file['data_type'], file['data_category']))


## Jupyter Test Question 2
### Find data from all patients who have been treated with "Radiation Therapy, NOS" and have both genomic and proteomic data.

<font color=blue> Find out what field has "Radiation Therapy, NOS" as a possible value<font>

In [15]:
unique_terms('ResearchSubject.Diagnosis.Treatment.type')

SELECT DISTINCT(_Treatment.type) FROM `gdc-bq-sample.cda_mvp.v3`, UNNEST(ResearchSubject) AS _ResearchSubject,UNNEST(_ResearchSubject.Diagnosis) AS _Diagnosis,UNNEST(_Diagnosis.Treatment) AS _Treatment ORDER BY _Treatment.type


[None,
 'Ablation, Radiofrequency',
 'Brachytherapy, High Dose',
 'Brachytherapy, Low Dose',
 'Chemotherapy',
 'Hormone Therapy',
 'Immunotherapy (Including Vaccines)',
 'Pharmaceutical Therapy, NOS',
 'Radiation Therapy, NOS',
 'Radiation, 3D Conformal',
 'Radiation, External Beam',
 'Radiation, Intensity-Modulated Radiotherapy',
 'Radiation, Proton Beam',
 'Stem Cell Transplantation, Allogeneic',
 'Stem Cell Transplantation, Autologous',
 'Stereotactic Radiosurgery',
 'Surgery',
 'Targeted Molecular Therapy']

<font color=blue>Just out of curiosity, how many systems are there?</font>

In [16]:
unique_terms('ResearchSubject.Specimen.identifier.system')

SELECT DISTINCT(_identifier.system) FROM `gdc-bq-sample.cda_mvp.v3`, UNNEST(ResearchSubject) AS _ResearchSubject,UNNEST(_ResearchSubject.Specimen) AS _Specimen,UNNEST(_Specimen.identifier) AS _identifier ORDER BY _identifier.system


['GDC', 'PDC']

<font color = blue> How many at each node individuallyl?</font>

In [17]:
treatment1 = Q('ResearchSubject.Diagnosis.Treatment.type = "Radiation Therapy, NOS"')
site1 = Q('ResearchSubject.Specimen.identifier.system = "GDC"')
site2 = Q('ResearchSubject.Specimen.identifier.system = "PDC"')

In [18]:
gdc = treatment1.And(site1)
r = gdc.run()
print(r)


Query: SELECT * FROM gdc-bq-sample.cda_mvp.v3, UNNEST(ResearchSubject) AS _ResearchSubject, UNNEST(_ResearchSubject.Diagnosis) AS _Diagnosis, UNNEST(_Diagnosis.Treatment) AS _Treatment, UNNEST(_ResearchSubject.Specimen) AS _Specimen, UNNEST(_Specimen.identifier) AS _identifier WHERE ((_Treatment.type = 'Radiation Therapy, NOS') AND (_identifier.system = 'GDC'))
Offset: 0
Limit: 1000
Count: 1000
More pages: Yes



<font color=blue> Clearly cases at GDC</font>

In [19]:
pdc = treatment1.And(site2)
r = pdc.run()
print(r)


Query: SELECT * FROM gdc-bq-sample.cda_mvp.v3, UNNEST(ResearchSubject) AS _ResearchSubject, UNNEST(_ResearchSubject.Diagnosis) AS _Diagnosis, UNNEST(_Diagnosis.Treatment) AS _Treatment, UNNEST(_ResearchSubject.Specimen) AS _Specimen, UNNEST(_Specimen.identifier) AS _identifier WHERE ((_Treatment.type = 'Radiation Therapy, NOS') AND (_identifier.system = 'PDC'))
Offset: 0
Limit: 1000
Count: 0
More pages: No



<font color=blue> And clearly no cases at PDC.  Which means we need to get all the cases at GDC and see if any match to PDC</font>

In [7]:
q1 = treatment1.And(site1)
r1 = q1.run()
print(r1)


Query: SELECT * FROM gdc-bq-sample.cda_mvp.v3, UNNEST(ResearchSubject) AS _ResearchSubject, UNNEST(_ResearchSubject.Diagnosis) AS _Diagnosis, UNNEST(_Diagnosis.Treatment) AS _Treatment, UNNEST(_ResearchSubject.Specimen) AS _Specimen, UNNEST(_Specimen.identifier) AS _identifier WHERE ((_Treatment.type = 'Radiation Therapy, NOS') AND (_identifier.system = 'GDC'))
Offset: 0
Limit: 1000
Count: 1000
More pages: Yes



In [8]:
q2 = site1.And(treatment1)
r2 = q2.run()
print(r2)


Query: SELECT * FROM gdc-bq-sample.cda_mvp.v3, UNNEST(ResearchSubject) AS _ResearchSubject, UNNEST(_ResearchSubject.Specimen) AS _Specimen, UNNEST(_Specimen.identifier) AS _identifier, UNNEST(_ResearchSubject.Diagnosis) AS _Diagnosis, UNNEST(_Diagnosis.Treatment) AS _Treatment WHERE ((_identifier.system = 'GDC') AND (_Treatment.type = 'Radiation Therapy, NOS'))
Offset: 0
Limit: 1000
Count: 1000
More pages: Yes



<font color=blue> A bug causes a 502 Bad Gateway error unless a limit is set</font>

In [15]:
q3 = site2.From(treatment1.And(site1))
limit = 500
r3 = q3.run(limit = limit)
print(r3)


Query: SELECT * FROM (SELECT * FROM gdc-bq-sample.cda_mvp.v3, UNNEST(ResearchSubject) AS _ResearchSubject, UNNEST(_ResearchSubject.Diagnosis) AS _Diagnosis, UNNEST(_Diagnosis.Treatment) AS _Treatment, UNNEST(_ResearchSubject.Specimen) AS _Specimen, UNNEST(_Specimen.identifier) AS _identifier WHERE ((_Treatment.type = 'Radiation Therapy, NOS') AND (_identifier.system = 'GDC'))), UNNEST(ResearchSubject) AS _ResearchSubject, UNNEST(_ResearchSubject.Specimen) AS _Specimen, UNNEST(_Specimen.identifier) AS _identifier WHERE (_identifier.system = 'PDC')
Offset: 0
Limit: 100
Count: 100
More pages: Yes



<font color = blue> So the query works, the question is how many patients there are, so need to loop through the results</font>

In [18]:
print(str(r3.next_page))

<bound method Result.next_page of <cdapython.Result object at 0x7faf409d0a60>>


In [ ]:
rscount = 0
while r3.count == limit:
    rscount = rscount + r3.count
    r3 = r3.next_page()
    print(("Current rscount: %s \tNext page: %s")%(str(rscount),r3.next_page))
    
# r3.count<100 it means we're on the last iteration, so just add it
rscount = rscount.r3.count

print(("Final Count: %s\n") % (str(rscount)))    

<font color = blue>Pulled the plug after it was at over 10K and showign no signs of stopping.  Maybe direct SQL to find out the count?</font>

In [25]:
query = (
    'SELECT COUNT(DISTINCT _ResearchSubject.id) as rsidcount FROM gdc-bq-sample.cda_mvp.v3,'
     'UNNEST(ResearchSubject) AS _ResearchSubject, UNNEST(_ResearchSubject.Diagnosis) AS _Diagnosis, UNNEST(_Diagnosis.Treatment) AS _Treatment '
     'WHERE _Treatment.type = "Radiation Therapy, NOS"'
)
result = runAPIQuery(query, None)
pprint.pprint(result)

{'next_url': None,
 'previous_url': None,
 'query_sql': 'SELECT COUNT(DISTINCT _ResearchSubject.id) as rsidcount FROM '
              'gdc-bq-sample.cda_mvp.v3,UNNEST(ResearchSubject) AS '
              '_ResearchSubject, UNNEST(_ResearchSubject.Diagnosis) AS '
              '_Diagnosis, UNNEST(_Diagnosis.Treatment) AS _Treatment WHERE '
              '_Treatment.type = "Radiation Therapy, NOS"',
 'result': [{'rsidcount': 11507}]}


<font color = blue>Well, almost 12K research subjects with this treatment, how many at GDC and PDC?</font>

In [28]:
gdcquery = (
    'SELECT COUNT(DISTINCT _ResearchSubject.id) as rsidcount FROM gdc-bq-sample.cda_mvp.v3, '
    'UNNEST(ResearchSubject) AS _ResearchSubject, UNNEST(_ResearchSubject.Diagnosis) AS _Diagnosis, UNNEST(_Diagnosis.Treatment) AS _Treatment, '
    'UNNEST(_ResearchSubject.Specimen) AS _Specimen, UNNEST(_Specimen.identifier) as _identifier WHERE _Treatment.type = "Radiation Therapy, NOS" '
    'AND _identifier.system = "GDC"'
)

pdcquery = (
    'SELECT COUNT(DISTINCT _ResearchSubject.id) as rsidcount FROM gdc-bq-sample.cda_mvp.v3, '
    'UNNEST(ResearchSubject) AS _ResearchSubject, UNNEST(_ResearchSubject.Diagnosis) AS _Diagnosis, UNNEST(_Diagnosis.Treatment) AS _Treatment, '
    'UNNEST(_ResearchSubject.Specimen) AS _Specimen, UNNEST(_Specimen.identifier) as _identifier WHERE _Treatment.type = "Radiation Therapy, NOS" '
    'AND _identifier.system = "PDC"'
)

gdcresult = runAPIQuery(gdcquery, None)
pprint.pprint(gdcresult)
pdcresult = runAPIQuery(pdcquery, None)
pprint.pprint(pdcresult)

{'next_url': None,
 'previous_url': None,
 'query_sql': 'SELECT COUNT(DISTINCT _ResearchSubject.id) as rsidcount FROM '
              'gdc-bq-sample.cda_mvp.v3, UNNEST(ResearchSubject) AS '
              '_ResearchSubject, UNNEST(_ResearchSubject.Diagnosis) AS '
              '_Diagnosis, UNNEST(_Diagnosis.Treatment) AS _Treatment, '
              'UNNEST(_ResearchSubject.Specimen) AS _Specimen, '
              'UNNEST(_Specimen.identifier) as _identifier WHERE '
              '_Treatment.type = "Radiation Therapy, NOS" AND '
              '_identifier.system = "GDC"',
 'result': [{'rsidcount': 11507}]}
{'next_url': None,
 'previous_url': None,
 'query_sql': 'SELECT COUNT(DISTINCT _ResearchSubject.id) as rsidcount FROM '
              'gdc-bq-sample.cda_mvp.v3, UNNEST(ResearchSubject) AS '
              '_ResearchSubject, UNNEST(_ResearchSubject.Diagnosis) AS '
              '_Diagnosis, UNNEST(_Diagnosis.Treatment) AS _Treatment, '
              'UNNEST(_ResearchSubject.Specimen) AS _

<font color=blue> Which confirms that all the cases are at GDC.  So, the question then becomes, so any of those ResearchSubject IDs have any record at PDC?</font>

In [29]:
comboquery = (
    'SELECT COUNT(DISTINCT _ResearchSubject.id) FROM (SELECT * FROM gdc-bq-sample.cda_mvp.v3, '
    'UNNEST(ResearchSubject) AS _ResearchSubject, UNNEST(_ResearchSubject.Diagnosis) AS _Diagnosis, UNNEST(_Diagnosis.Treatment) AS _Treatment, '
    'UNNEST(_ResearchSubject.Specimen) AS _Specimen, UNNEST(_Specimen.identifier) as _identifier '
    'WHERE ((_Treatment.type = "Radiation Therapy, NOS") AND (_identifier.system = "GDC"))), '
    'UNNEST(ResearchSubject) AS _ResearchSubject, UNNEST(_ResearchSubject.identifier) AS _identifier WHERE (_identifier.system = "PDC")'
)

comboresult = runAPIQuery(comboquery, None)
pprint.pprint(comboresult)

{'next_url': None,
 'previous_url': None,
 'query_sql': 'SELECT COUNT(DISTINCT _ResearchSubject.id) FROM (SELECT * FROM '
              'gdc-bq-sample.cda_mvp.v3, UNNEST(ResearchSubject) AS '
              '_ResearchSubject, UNNEST(_ResearchSubject.Diagnosis) AS '
              '_Diagnosis, UNNEST(_Diagnosis.Treatment) AS _Treatment, '
              'UNNEST(_ResearchSubject.Specimen) AS _Specimen, '
              'UNNEST(_Specimen.identifier) as _identifier WHERE '
              '((_Treatment.type = "Radiation Therapy, NOS") AND '
              '(_identifier.system = "GDC"))), UNNEST(ResearchSubject) AS '
              '_ResearchSubject, UNNEST(_ResearchSubject.identifier) AS '
              '_identifier WHERE (_identifier.system = "PDC")',
 'result': [{'': 369}]}


<font color = blue> So it looks like there are 369 research subjects that have data in both GDC and PDC.  I'm hoping this is what would be seen if the bad gateway error hadn't been present</font>

# Test Query 3
## Find data from all subjects with lung adenocarcinomas who have both primary and recurrent tumors

<font color=blue> Find out where "lung adenocarciomas" is a valid value

In [2]:
unique_terms('ResearchSubject.Specimen.primary_disease_type')

SELECT DISTINCT(_Specimen.primary_disease_type) FROM `gdc-bq-sample.cda_mvp.v3`, UNNEST(ResearchSubject) AS _ResearchSubject,UNNEST(_ResearchSubject.Specimen) AS _Specimen ORDER BY _Specimen.primary_disease_type


[None,
 'Acinar Cell Neoplasms',
 'Adenomas and Adenocarcinomas',
 'Adnexal and Skin Appendage Neoplasms',
 'Basal Cell Neoplasms',
 'Blood Vessel Tumors',
 'Breast Invasive Carcinoma',
 'Chromophobe Renal Cell Carcinoma',
 'Chronic Myeloproliferative Disorders',
 'Clear Cell Renal Cell Carcinoma',
 'Colon Adenocarcinoma',
 'Complex Epithelial Neoplasms',
 'Complex Mixed and Stromal Neoplasms',
 'Cystic, Mucinous and Serous Neoplasms',
 'Ductal and Lobular Neoplasms',
 'Early Onset Gastric Cancer',
 'Epithelial Neoplasms, NOS',
 'Fibroepithelial Neoplasms',
 'Fibromatous Neoplasms',
 'Germ Cell Neoplasms',
 'Giant Cell Tumors',
 'Glioblastoma',
 'Gliomas',
 'Granular Cell Tumors and Alveolar Soft Part Sarcomas',
 'Head and Neck Squamous Cell Carcinoma',
 'Hepatocellular Carcinoma ',
 'Hodgkin Lymphoma',
 'Immunoproliferative Diseases',
 'Leukemias, NOS',
 'Lipomatous Neoplasms',
 'Lung Adenocarcinoma',
 'Lung Squamous Cell Carcinoma',
 'Lymphatic Vessel Tumors',
 'Lymphoid Leukemias',


In [3]:
unique_terms('ResearchSubject.primary_disease_type')

SELECT DISTINCT(_ResearchSubject.primary_disease_type) FROM `gdc-bq-sample.cda_mvp.v3`, UNNEST(ResearchSubject) AS _ResearchSubject ORDER BY _ResearchSubject.primary_disease_type


[None,
 'Acinar Cell Neoplasms',
 'Adenomas and Adenocarcinomas',
 'Adnexal and Skin Appendage Neoplasms',
 'Basal Cell Neoplasms',
 'Blood Vessel Tumors',
 'Breast Invasive Carcinoma',
 'Chromophobe Renal Cell Carcinoma',
 'Chronic Myeloproliferative Disorders',
 'Clear Cell Renal Cell Carcinoma',
 'Colon Adenocarcinoma',
 'Complex Epithelial Neoplasms',
 'Complex Mixed and Stromal Neoplasms',
 'Cystic, Mucinous and Serous Neoplasms',
 'Ductal and Lobular Neoplasms',
 'Early Onset Gastric Cancer',
 'Epithelial Neoplasms, NOS',
 'Fibroepithelial Neoplasms',
 'Fibromatous Neoplasms',
 'Germ Cell Neoplasms',
 'Giant Cell Tumors',
 'Glioblastoma',
 'Gliomas',
 'Granular Cell Tumors and Alveolar Soft Part Sarcomas',
 'Head and Neck Squamous Cell Carcinoma',
 'Hepatocellular Carcinoma ',
 'Hodgkin Lymphoma',
 'Immunoproliferative Diseases',
 'Leukemias, NOS',
 'Lipomatous Neoplasms',
 'Lung Adenocarcinoma',
 'Lung Squamous Cell Carcinoma',
 'Lymphatic Vessel Tumors',
 'Lymphoid Leukemias',


<font color=blue>  So this exists in both ResearchSubject.primary_disease_type and ResearchSubject.Sample.primary_disease_type.  Since we're intersted in patients, not samples, using ResearchSubject.primary_disease_type seems appropriate.</font>

<font color=blue> So the next step is to figure out where primary and recurring tumors are identified.  There's nothign at the ResearchSubject level however Sample has source_material_type which seems likely</font>

In [30]:
unique_terms('ResearchSubject.Specimen.source_material_type')

SELECT DISTINCT(_Specimen.source_material_type) FROM `gdc-bq-sample.cda_mvp.v3`, UNNEST(ResearchSubject) AS _ResearchSubject,UNNEST(_ResearchSubject.Specimen) AS _Specimen ORDER BY _Specimen.source_material_type


['Additional - New Primary',
 'Additional Metastatic',
 'Blood Derived Cancer - Bone Marrow',
 'Blood Derived Cancer - Bone Marrow, Post-treatment',
 'Blood Derived Cancer - Peripheral Blood',
 'Blood Derived Cancer - Peripheral Blood, Post-treatment',
 'Blood Derived Normal',
 'Bone Marrow Normal',
 'Buccal Cell Normal',
 'Cell Lines',
 'Control Analyte',
 'DNA',
 'Expanded Next Generation Cancer Model',
 'FFPE Scrolls',
 'Fibroblasts from Bone Marrow Normal',
 'Granulocytes',
 'Lymphoid Normal',
 'Metastatic',
 'Mononuclear Cells from Bone Marrow Normal',
 'Neoplasms of Uncertain and Unknown Behavior',
 'Next Generation Cancer Model',
 'Normal',
 'Normal Adjacent Tissue',
 'Not Reported',
 'Post neo-adjuvant therapy',
 'Primary Blood Derived Cancer - Bone Marrow',
 'Primary Blood Derived Cancer - Peripheral Blood',
 'Primary Tumor',
 'Primary Xenograft Tissue',
 'Recurrent Blood Derived Cancer - Bone Marrow',
 'Recurrent Blood Derived Cancer - Peripheral Blood',
 'Recurrent Tumor',
 

<font color=blue> Clearly there are several options, however "Primary Tumor" and "Recurrent Tumor" seem the most generic</font>

In [4]:
tumor1 = Q('ResearchSubject.Specimen.source_material_type = "Primary Tumor"')
tumor2 = Q('ResearchSubject.Specimen.source_material_type = "Recurrent Tumor"')

disease = Q('ResearchSubject.primary_disease_type = "Lung Adenocarcinoma"')

In [5]:
rd= disease.run()
print(rd)


Query: SELECT * FROM gdc-bq-sample.cda_mvp.v3, UNNEST(ResearchSubject) AS _ResearchSubject WHERE (_ResearchSubject.primary_disease_type = 'Lung Adenocarcinoma')
Offset: 0
Limit: 1000
Count: 216
More pages: No



<font color=blue>Looks like 216 ReseachSubjects labeled as Lung Adenocarcinoma</font>

In [6]:
q = disease.And(tumor1)
r = q.run()
print(r)


Query: SELECT * FROM gdc-bq-sample.cda_mvp.v3, UNNEST(ResearchSubject) AS _ResearchSubject, UNNEST(_ResearchSubject.Specimen) AS _Specimen WHERE ((_ResearchSubject.primary_disease_type = 'Lung Adenocarcinoma') AND (_Specimen.source_material_type = 'Primary Tumor'))
Offset: 0
Limit: 1000
Count: 121
More pages: No



<font color=blue> A touch over half are primary tumor</font>

In [7]:
q = disease.And(tumor2)
r = q.run()
print(r)


Query: SELECT * FROM gdc-bq-sample.cda_mvp.v3, UNNEST(ResearchSubject) AS _ResearchSubject, UNNEST(_ResearchSubject.Specimen) AS _Specimen WHERE ((_ResearchSubject.primary_disease_type = 'Lung Adenocarcinoma') AND (_Specimen.source_material_type = 'Recurrent Tumor'))
Offset: 0
Limit: 1000
Count: 0
More pages: No



<font color=blue>Huh.  None of them are recurrent.  Which raises the question, are there ANY recurrent tumors in the system</font>

In [8]:
r = tumor2.run()
print(r)


Query: SELECT * FROM gdc-bq-sample.cda_mvp.v3, UNNEST(ResearchSubject) AS _ResearchSubject, UNNEST(_ResearchSubject.Specimen) AS _Specimen WHERE (_Specimen.source_material_type = 'Recurrent Tumor')
Offset: 0
Limit: 1000
Count: 1000
More pages: Yes



<font color=blue> Clearly lots of recurrent tumors, just not from lung adenocarcinoma.  So using a FROM query should identify any cases with recurrent tumors from something else?</font>

In [9]:
q = tumor2.From(disease.And(tumor1))
r = q.run()
print(r)


Query: SELECT * FROM (SELECT * FROM gdc-bq-sample.cda_mvp.v3, UNNEST(ResearchSubject) AS _ResearchSubject, UNNEST(_ResearchSubject.Specimen) AS _Specimen WHERE ((_ResearchSubject.primary_disease_type = 'Lung Adenocarcinoma') AND (_Specimen.source_material_type = 'Primary Tumor'))), UNNEST(ResearchSubject) AS _ResearchSubject, UNNEST(_ResearchSubject.Specimen) AS _Specimen WHERE (_Specimen.source_material_type = 'Recurrent Tumor')
Offset: 0
Limit: 1000
Count: 0
More pages: No



<font color=blue>That was a bust.  Maybe first finding cases that have both primary and recurring first is a better approach</font>

In [10]:
q = tumor1.And(tumor2)
r = q.run()
print(r)


Query: SELECT * FROM gdc-bq-sample.cda_mvp.v3, UNNEST(ResearchSubject) AS _ResearchSubject, UNNEST(_ResearchSubject.Specimen) AS _Specimen WHERE ((_Specimen.source_material_type = 'Primary Tumor') AND (_Specimen.source_material_type = 'Recurrent Tumor'))
Offset: 0
Limit: 1000
Count: 0
More pages: No



<font color=blue> So there are no instances of both primary and recurring.  Let's try a different approach</font>

In [31]:
query = (
    'SELECT COUNT(DISTINCT _ResearchSubject.id) AS rsid FROM (SELECT * FROM gdc-bq-sample.cda_mvp.v3, UNNEST(ResearchSubject) AS _ResearchSubject, '
    'UNNEST(_ResearchSubject.Specimen) AS _Specimen WHERE ((_ResearchSubject.primary_disease_type = "Lung Adenocarcinoma") '
    'AND (_Specimen.source_material_type = "Primary Tumor"))), UNNEST(ResearchSubject) AS _ResearchSubject, '
    'UNNEST(_ResearchSubject.Specimen) AS _Specimen WHERE (_Specimen.source_material_type = "Recurrent Tumor")'
)

result = runAPIQuery(query, None)
pprint.pprint(result)

{'next_url': None,
 'previous_url': None,
 'query_sql': 'SELECT COUNT(DISTINCT _ResearchSubject.id) AS rsid FROM (SELECT '
              '* FROM gdc-bq-sample.cda_mvp.v3, UNNEST(ResearchSubject) AS '
              '_ResearchSubject, UNNEST(_ResearchSubject.Specimen) AS '
              '_Specimen WHERE ((_ResearchSubject.primary_disease_type = "Lung '
              'Adenocarcinoma") AND (_Specimen.source_material_type = "Primary '
              'Tumor"))), UNNEST(ResearchSubject) AS _ResearchSubject, '
              'UNNEST(_ResearchSubject.Specimen) AS _Specimen WHERE '
              '(_Specimen.source_material_type = "Recurrent Tumor")',
 'result': [{'rsid': 0}]}


<font color=blue> Apparently time to look at the other recurrents and see if those exist</font>

In [32]:
recurrents = ['Recurrent Blood Derived Cancer - Bone Marrow',
 'Recurrent Blood Derived Cancer - Peripheral Blood',
 'Recurrent Tumor',]


for recurrent in recurrents:
    querystring = 'ResearchSubject.Specimen.source_material_type = "{}"'.format(recurrent)
    query = Q(querystring)
    r = query.run()
    print(recurrent)
    print(r)

Recurrent Blood Derived Cancer - Bone Marrow

Query: SELECT * FROM gdc-bq-sample.cda_mvp.v3, UNNEST(ResearchSubject) AS _ResearchSubject, UNNEST(_ResearchSubject.Specimen) AS _Specimen WHERE (_Specimen.source_material_type = 'Recurrent Blood Derived Cancer - Bone Marrow')
Offset: 0
Limit: 1000
Count: 1000
More pages: Yes

Recurrent Blood Derived Cancer - Peripheral Blood

Query: SELECT * FROM gdc-bq-sample.cda_mvp.v3, UNNEST(ResearchSubject) AS _ResearchSubject, UNNEST(_ResearchSubject.Specimen) AS _Specimen WHERE (_Specimen.source_material_type = 'Recurrent Blood Derived Cancer - Peripheral Blood')
Offset: 0
Limit: 1000
Count: 1000
More pages: Yes

Recurrent Tumor

Query: SELECT * FROM gdc-bq-sample.cda_mvp.v3, UNNEST(ResearchSubject) AS _ResearchSubject, UNNEST(_ResearchSubject.Specimen) AS _Specimen WHERE (_Specimen.source_material_type = 'Recurrent Tumor')
Offset: 0
Limit: 1000
Count: 1000
More pages: Yes



<font color=blue>So all are potentially valid, lets take a look</font>

In [35]:
for recurrent in recurrents: 
    query = (
    'SELECT COUNT(DISTINCT _ResearchSubject.id) AS rsid FROM (SELECT * FROM gdc-bq-sample.cda_mvp.v3, UNNEST(ResearchSubject) AS _ResearchSubject, '
    'UNNEST(_ResearchSubject.Specimen) AS _Specimen WHERE ((_ResearchSubject.primary_disease_type = "Lung Adenocarcinoma") '
    'AND (_Specimen.source_material_type = "Primary Tumor"))), UNNEST(ResearchSubject) AS _ResearchSubject, '
    'UNNEST(_ResearchSubject.Specimen) AS _Specimen WHERE (_Specimen.source_material_type = "{}")'
    ).format(recurrent)
    result = runAPIQuery(query, None)
    print(recurrent)
    pprint.pprint(result)


Recurrent Blood Derived Cancer - Bone Marrow
{'next_url': None,
 'previous_url': None,
 'query_sql': 'SELECT COUNT(DISTINCT _ResearchSubject.id) AS rsid FROM (SELECT '
              '* FROM gdc-bq-sample.cda_mvp.v3, UNNEST(ResearchSubject) AS '
              '_ResearchSubject, UNNEST(_ResearchSubject.Specimen) AS '
              '_Specimen WHERE ((_ResearchSubject.primary_disease_type = "Lung '
              'Adenocarcinoma") AND (_Specimen.source_material_type = "Primary '
              'Tumor"))), UNNEST(ResearchSubject) AS _ResearchSubject, '
              'UNNEST(_ResearchSubject.Specimen) AS _Specimen WHERE '
              '(_Specimen.source_material_type = "Recurrent Blood Derived '
              'Cancer - Bone Marrow")',
 'result': [{'rsid': 0}]}
Recurrent Blood Derived Cancer - Peripheral Blood
{'next_url': None,
 'previous_url': None,
 'query_sql': 'SELECT COUNT(DISTINCT _ResearchSubject.id) AS rsid FROM (SELECT '
              '* FROM gdc-bq-sample.cda_mvp.v3, UNNEST(Research

<font color=blue>Looks like no choice but to go full brute force</font>

In [36]:
primaries = ['Primary Blood Derived Cancer - Bone Marrow',
 'Primary Blood Derived Cancer - Peripheral Blood',
 'Primary Tumor',]

for primary in primaries:
    for recurrent in recurrents:
            query = (
    'SELECT COUNT(DISTINCT _ResearchSubject.id) AS rsid FROM (SELECT * FROM gdc-bq-sample.cda_mvp.v3, UNNEST(ResearchSubject) AS _ResearchSubject, '
    'UNNEST(_ResearchSubject.Specimen) AS _Specimen WHERE ((_ResearchSubject.primary_disease_type = "Lung Adenocarcinoma") '
    'AND (_Specimen.source_material_type = "{}"))), UNNEST(ResearchSubject) AS _ResearchSubject, '
    'UNNEST(_ResearchSubject.Specimen) AS _Specimen WHERE (_Specimen.source_material_type = "{}")'
    ).format(primary, recurrent)
            result = runAPIQuery(query, None)
            print(("Primary: %s\tRecurrent: %s\n") % (primary, recurrent))
            pprint.pprint(result)

Primary: Primary Blood Derived Cancer - Bone Marrow	Recurrent: Recurrent Blood Derived Cancer - Bone Marrow

{'next_url': None,
 'previous_url': None,
 'query_sql': 'SELECT COUNT(DISTINCT _ResearchSubject.id) AS rsid FROM (SELECT '
              '* FROM gdc-bq-sample.cda_mvp.v3, UNNEST(ResearchSubject) AS '
              '_ResearchSubject, UNNEST(_ResearchSubject.Specimen) AS '
              '_Specimen WHERE ((_ResearchSubject.primary_disease_type = "Lung '
              'Adenocarcinoma") AND (_Specimen.source_material_type = "Primary '
              'Blood Derived Cancer - Bone Marrow"))), UNNEST(ResearchSubject) '
              'AS _ResearchSubject, UNNEST(_ResearchSubject.Specimen) AS '
              '_Specimen WHERE (_Specimen.source_material_type = "Recurrent '
              'Blood Derived Cancer - Bone Marrow")',
 'result': [{'rsid': 0}]}
Primary: Primary Blood Derived Cancer - Bone Marrow	Recurrent: Recurrent Blood Derived Cancer - Peripheral Blood

{'next_url': None,
 'previous

<font color=blue>Looks to me like there are no patients that fit the test question</font>